In [1]:
from cfg import read_grammar_rules
from cfg import WCFG
from rule import Rule

# Input

In this example we will parse a grammar that performs arithmetic operations such as sum and multiplication.

In [19]:
istream = open('examples/arithmetic', 'r')


In [20]:
rules = list(read_grammar_rules(istream))

In [21]:
rules[0]

[E] -> [T] (0.5)

In [22]:
G = WCFG(rules)

In [23]:
print G

[T] -> [P] (0.5)
[T] -> [T] * [P] (0.5)
[E] -> [T] (0.5)
[E] -> [E] + [T] (0.4)
[E] -> [T] + [E] (0.1)
[P] -> a (1.0)


# Parser

We are using dotted rules to represent items in CKY and Earley.

A dotted rule looks like the following:

In [24]:
class Item(object):
    
    def __init__(self, rule, dots):
        assert len(dots) > 0, 'I do not accept an empty list of dots'
        self.rule_ = rule
        self.dots_ = tuple(dots)
        
    def __eq__(self, other):
        return self.rule_ == other.rule_ and self.dots_ == other.dots_
    
    def __ne__(self, other):
        return not(self == other)
    
    def __hash__(self):
        return hash((self.rule_, self.dots_))
    
    def __repr__(self):
        return '{0} ||| {1}'.format(self.rule_, self.dots_)
    
    def __str__(self):
        return '{0} ||| {1}'.format(self.rule_, self.dots_)
    
    @property
    def lhs(self):
        return self.rule_.lhs
    
    @property
    def rule(self):
        return self.rule_
    
    @property
    def dot(self):
        return self.dots_[-1]
    
    @property
    def start(self):
        return self.dots_[0]
    
    def advance(self, dot):
        """return a new item with an extended sequence of dots"""
        return Item(self.rule_, self.dots_ + (dot,))
    
    def is_complete(self):
        """complete items are those whose dot reached the end of the RHS sequence"""
        return len(self.rule_.rhs) + 1 == len(self.dots_)
    
    def next(self):
        if self.is_complete():
            return None
        return self.rule_.rhs[len(self.dots_) - 1]
    
    

In [25]:
r = Rule('[S]', ['[X]'], 0.0)

In [26]:
r

[S] -> [X] (0.0)

In [27]:
i1 = Item(r, [0])

In [28]:
i2 = i1.advance(1)

In [29]:
i1, i2

([S] -> [X] (0.0) ||| (0,), [S] -> [X] (0.0) ||| (0, 1))

In [30]:
i1 != i2

True

In [31]:
i1 == i2

False

In [32]:
i1.is_complete()


False

In [33]:
i2.is_complete()

True

In [17]:
i1.next()

'[X]'

In [18]:
i2.next()

## Axioms (CKY)

For every rule X -> alpha, and every input position (i) between 0 and n-1, we have an item of the kind:

* [X -> * alpha, i, i]

which gets represented as

* X -> alpha, [i]

In [41]:
def axioms(cfg, sentence):
    """
    :params cfg: a context-free grammar (an instance of WCFG)
    :params sentence: the input sentence (as a list or tuple)
    :returns: a list of items
    """
    items = []
    for rule in cfg:
        for i in range(len(sentence)):
            items.append(Item(rule, [i]))
    return items

In [43]:
sentence = 'a * a'.split()
axioms(G, sentence)

[[E] -> [T] (0.5) ||| (0,),
 [E] -> [T] (0.5) ||| (1,),
 [E] -> [T] (0.5) ||| (2,),
 [E] -> [E] + [T] (0.4) ||| (0,),
 [E] -> [E] + [T] (0.4) ||| (1,),
 [E] -> [E] + [T] (0.4) ||| (2,),
 [E] -> [T] + [E] (0.1) ||| (0,),
 [E] -> [T] + [E] (0.1) ||| (1,),
 [E] -> [T] + [E] (0.1) ||| (2,),
 [T] -> [P] (0.5) ||| (0,),
 [T] -> [P] (0.5) ||| (1,),
 [T] -> [P] (0.5) ||| (2,),
 [T] -> [T] * [P] (0.5) ||| (0,),
 [T] -> [T] * [P] (0.5) ||| (1,),
 [T] -> [T] * [P] (0.5) ||| (2,),
 [P] -> a (1.0) ||| (0,),
 [P] -> a (1.0) ||| (1,),
 [P] -> a (1.0) ||| (2,)]